In [6]:
# !pip install openai==0.28.1
!pip install fpdf==1.7.2

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40713 sha256=431c8e9d4bdb0ebcb24dc274c28a3fba001c8a2f6b9fea0b52cceabed756014f
  Stored in directory: c:\users\lrinc\appdata\local\pip\cache\wheels\6e\62\11\dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [7]:
import pandas as pd
import openai
import time
from fpdf import FPDF

openai.api_key = ''

## Load the dataset

In [10]:
df = pd.read_csv('https://raw.githubusercontent.com/xlaw1x/Day-4-AI-First-Dataset-Live/refs/heads/main/Agrikonek%20AI.csv', encoding='latin1')

In [12]:
df.sample(5)

,Shipment_ID,Farmer_Name,Product_Type,Weight_kg,Origin,Destination,Transport_Type,Temp_Control,Packaging_Type,Traffic_Condition,Season,Distance_km,Estimated_Transit_Hours,Cost_PHP,Notes
16,S017,Juanito de la Cruz,Mais,220,Quezon Province,Manila,Open Air Truck,No,Burlap Bags,Moderate,Humid,130,4.5,2800,"Needs breathable packaging, frequent airing"
5,S006,Lito Bautista,Repolyo,250,Nueva Vizcaya,Caloocan City,Refrigerated Truck,Yes,Cardboard + Wrapping,Moderate,Wet Season,230,9.0,3800,Harvested early for freshness
13,S014,Mila Hernandez,Malunggay,20,Batangas,Makati City,Open Air Truck,No,Lightweight Bundles,Light,Hot,110,2.5,900,"Delicate, prone to wilting"
0,S001,Ramon Santos,Kamatis,150,Benguet,Metro Manila,Refrigerated Truck,Yes,Ventilated Crates,Heavy,Rainy,250,10.0,3500,Peak season for tomatoes
3,S004,Victor Flores,Bawang,100,Ilocos Norte,Pasig City,Open Air Truck,No,Mesh Bags,Heavy,Rainy,330,12.0,4000,Odor control needed


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Shipment_ID              20 non-null     object 
 1   Farmer_Name              20 non-null     object 
 2   Product_Type             20 non-null     object 
 3   Weight_kg                20 non-null     int64  
 4   Origin                   20 non-null     object 
 5   Destination              20 non-null     object 
 6   Transport_Type           20 non-null     object 
 7   Temp_Control             20 non-null     object 
 8   Packaging_Type           20 non-null     object 
 9   Traffic_Condition        20 non-null     object 
 10  Season                   20 non-null     object 
 11  Distance_km              20 non-null     int64  
 12  Estimated_Transit_Hours  20 non-null     float64
 13  Cost_PHP                 20 non-null     int64  
 14  Notes                    20 

## Understanding the Dataset

In [18]:
# Get Column Names and Data Types
column_info = df.dtypes
column_info

Shipment_ID                 object
Farmer_Name                 object
Product_Type                object
Weight_kg                    int64
Origin                      object
Destination                 object
Transport_Type              object
Temp_Control                object
Packaging_Type              object
Traffic_Condition           object
Season                      object
Distance_km                  int64
Estimated_Transit_Hours    float64
Cost_PHP                     int64
Notes                       object
dtype: object

In [20]:
# Identify Categorical and Numerical Features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64', 'float']).columns

In [22]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display=10):
    uniques = df[col].unique().tolist() # get unique values and turn them intro lists
    if len(uniques) > max_display: # if there are more than 10 unique values
        return uniques[:max_display] + ["..."]

unique_values = {col: get_unique_values(col) for col in categorical_features}

print("Unique Values for Categorical Features:")
for col, uniques in unique_values.items():
    print(f" - {col}: {uniques}")
print()

Unique Values for Categorical Features:
 - Shipment_ID: ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', '...']
 - Farmer_Name: ['Ramon Santos', 'Linda Cruz', 'Benjie Reyes', 'Victor Flores', 'Carding Mendoza', 'Lito Bautista', 'Maria Navarro', 'Isagani Ramos', 'Pepe Gonzales', 'Pilar Villanueva', '...']
 - Product_Type: ['Kamatis', 'Mangga', 'Bangus', 'Bawang', 'Saging', 'Repolyo', 'Pineapple', 'Langka', 'Pandesal', 'Kalabasa', '...']
 - Origin: ['Benguet', 'Zambales', 'Dagupan City', 'Ilocos Norte', 'Davao City', 'Nueva Vizcaya', 'Bukidnon', 'Laguna', 'Pampanga', 'Tarlac', '...']
 - Destination: ['Metro Manila', 'Quezon City', 'Makati City', 'Pasig City', 'Cebu City', 'Caloocan City', 'Pasay City', 'Taguig City', 'Mandaluyong City', 'Manila', '...']
 - Transport_Type: None
 - Temp_Control: None
 - Packaging_Type: ['Ventilated Crates', 'Sturdy Cardboard Box', 'Insulated Coolers', 'Mesh Bags', 'Sturdy Crates', 'Cardboard + Wrapping', 'Cardboard Boxes', '

In [30]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = df[numerical_features].describe()
print("Descriptive Statistics for Numerical Features:\n", numerical_stats, "\n")


Descriptive Statistics for Numerical Features:
         Weight_kg  Distance_km  Estimated_Transit_Hours     Cost_PHP
count   20.000000    20.000000                20.000000    20.000000
mean   163.750000   260.000000                 8.225000  3900.000000
std     79.981495   202.796242                 5.838788  2625.382178
min     20.000000    40.000000                 1.000000   900.000000
25%    100.000000   107.500000                 3.375000  1900.000000
50%    165.000000   180.000000                 6.500000  3300.000000
75%    220.000000   387.500000                12.750000  5300.000000
max    300.000000   620.000000                18.000000  9000.000000 



## Generating the Template

In [36]:
def generate_template(df, 
                      column_info, 
                      categorical_features, 
                      numerical_features, 
                      unique_values, 
                      numerical_stats):

    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for col, dtype in column_info.items():
        column_summary += f" - {col}: {dtype}\n"


    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features:\n"
    for col, uniques in unique_values.items():
        unique_values_str += f" - {col}: {uniques}\n"


    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics for Numerical Features:\n"
    for col in numerical_features:
        numerical_stats_str += f" - {col}:\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str + f" - {stat_name}: {value}\n"


    # Define the system prompt
    system_prompt = """You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs. 
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column. 
    Ensure the template is contextually relevant and maintains grammatical correctness."""



    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."

    
    Output only the template without any explanation or introduction.
    The template's variables will be dynamically replaced so make sure they're formatted properly."""


    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.3,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}") # e is error log
            time.sleep(2)  # Wait before retrying

    return None


In [38]:
# Generate the template
template = generate_template(df, 
                             column_info, 
                             categorical_features, 
                             numerical_features, 
                             unique_values, 
                             numerical_stats)

print(template)

"In shipment {Shipment_ID}, farmer {Farmer_Name} transported {Weight_kg} kg of {Product_Type} from {Origin} to {Destination} using {Transport_Type}. The product was packaged in {Packaging_Type} and required {Temp_Control} during transit. Given the {Traffic_Condition} conditions, the estimated distance of {Distance_km} km would take approximately {Estimated_Transit_Hours} hours to reach its destination at a cost of {Cost_PHP} PHP. Additional notes include: {Notes}."


## Populating the Template with Actual Values

In [44]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A' ---transform every row into dictionary
    row_dict = row.to_dict()
    for key, value in row_dict.items():
        if pd.isna(value):
            row_dict[key] = 'N/A'

    # Generate the populated template per row 
    paragraph = template.format(**row_dict)

    return paragraph

In [46]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)


In [48]:
df['paragraph'][0]

'"In shipment S001, farmer Ramon Santos transported 150 kg of Kamatis from Benguet to Metro Manila using Refrigerated Truck. The product was packaged in Ventilated Crates and required Yes during transit. Given the Heavy conditions, the estimated distance of 250 km would take approximately 10.0 hours to reach its destination at a cost of 3500 PHP. Additional notes include: Peak season for tomatoes."'

In [50]:
df.sample(5)

,Shipment_ID,Farmer_Name,Product_Type,Weight_kg,Origin,Destination,Transport_Type,Temp_Control,Packaging_Type,Traffic_Condition,Season,Distance_km,Estimated_Transit_Hours,Cost_PHP,Notes,paragraph
12,S013,Tony Castillo,Talong,180,Cavite,Quezon City,Refrigerated Truck,Yes,Plastic Wrapping,Heavy,Wet Season,40,1.0,1600,"Harvested recently, handle gently","""In shipment S013, farmer Tony Castillo transp..."
2,S003,Benjie Reyes,Bangus,300,Dagupan City,Makati City,Refrigerated Van,Yes,Insulated Coolers,Light,Hot/Dry,210,8.0,4800,Strict temperature control,"""In shipment S003, farmer Benjie Reyes transpo..."
8,S009,Pepe Gonzales,Pandesal,50,Pampanga,Makati City,Non-Refrigerated,No,Moisture-Control Bags,Light,Hot,90,2.0,1200,Ensure bags stay dry,"""In shipment S009, farmer Pepe Gonzales transp..."
16,S017,Juanito de la Cruz,Mais,220,Quezon Province,Manila,Open Air Truck,No,Burlap Bags,Moderate,Humid,130,4.5,2800,"Needs breathable packaging, frequent airing","""In shipment S017, farmer Juanito de la Cruz t..."
7,S008,Isagani Ramos,Langka,100,Laguna,Pasay City,Non-Refrigerated,No,Cardboard Boxes,Heavy,Dry,80,3.0,1500,Transport immediately to avoid overripening,"""In shipment S008, farmer Isagani Ramos transp..."


## Saving the Populated Templates (Natural Language Texts)

In [64]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:
            
            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation
        
        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")


In [66]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)
        
        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs
        
        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")

In [68]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['paragraph'].to_list(),
                    txt_filename ='nlg.txt')
save_content_to_pdf(df['paragraph'].to_list(),
                   pdf_filename= 'nlg.pdf')


All content has been saved as 'nlg.txt'.

All content has been saved as 'nlg.pdf'.

